In [1]:
import pandas as pd 
#import re
from sqlalchemy import create_engine
##import pyodbc

#### Read sales date data from stg_date

#### Create a connection to SQL Srever 

In [2]:
server = 'DB_SERVER'
database = 'DB_NAME'
#username = ''
#password = '
conn_str = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)
engine

Engine(mssql+pyodbc://@DB_SERVER/DB_NAME?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes)

#### Read Sales Date Data from stg_date table

In [3]:
sql_stg_date = '''SELECT *
                  FROM  stg_date'''

In [4]:
# store data to a dataframe 
stg_date_df = pd.read_sql(sql_stg_date,engine)
stg_date_df.head()

,order_date,qtr,month,year
0,2003-02-24,1,2,2003
1,2003-05-07,2,5,2003
2,2003-07-01,3,7,2003
3,2003-08-25,3,8,2003
4,2003-10-10,4,10,2003


In [5]:
len(stg_date_df)

2823

#### Cleand Sales Date Data 

In [6]:
stg_date_df['order_date'] = pd.to_datetime(stg_date_df['order_date'])
stg_date_df['date_id'] = stg_date_df['order_date'].dt.strftime('%Y%m%d').astype(int)

In [7]:
stg_date_df['date_id']

0       20030224
1       20030507
2       20030701
3       20030825
4       20031010
          ...   
2818    20041202
2819    20050131
2820    20050301
2821    20050328
2822    20050506
Name: date_id, Length: 2823, dtype: int64

In [8]:
# check distinict order_date values 
stg_date_df['date_id'].value_counts()

date_id
20031114    38
20041124    35
20031112    34
20041117    32
20041104    29
            ..
20040420     1
20030411     1
20050302     1
20030813     1
20030421     1
Name: count, Length: 252, dtype: int64

In [9]:
# Check for null values 
stg_date_df['date_id'].isnull().sum()


np.int64(0)

##### Drop duplicates

In [10]:
# Drop duplicates
stg_date_df= stg_date_df.drop_duplicates()

In [11]:
len(stg_date_df)

252

In [12]:
stg_date_df.head(10)

,order_date,qtr,month,year,date_id
0,2003-02-24,1,2,2003,20030224
1,2003-05-07,2,5,2003,20030507
2,2003-07-01,3,7,2003,20030701
3,2003-08-25,3,8,2003,20030825
4,2003-10-10,4,10,2003,20031010
5,2003-10-28,4,10,2003,20031028
6,2003-11-11,4,11,2003,20031111
7,2003-11-18,4,11,2003,20031118
8,2003-12-01,4,12,2003,20031201
9,2004-01-15,1,1,2004,20040115


In [13]:
stg_date_df = stg_date_df.copy()
stg_date_df = stg_date_df.reset_index(drop = True)
stg_date_df

,order_date,qtr,month,year,date_id
0,2003-02-24,1,2,2003,20030224
1,2003-05-07,2,5,2003,20030507
2,2003-07-01,3,7,2003,20030701
3,2003-08-25,3,8,2003,20030825
4,2003-10-10,4,10,2003,20031010
...,...,...,...,...,...
247,2003-04-11,2,4,2003,20030411
248,2003-08-13,3,8,2003,20030813
249,2003-10-08,4,10,2003,20031008
250,2005-03-28,1,3,2005,20050328


#### Load Data to DimDate Using Incremental Load Approach

##### 1) Load Sales Date Data from DimDate Table

In [14]:
# load DimDate table into a DataFrame
sql_DimDate = '''SELECT *
                  FROM  DimDate'''

In [15]:
dim_date_df = pd.read_sql(sql_DimDate, engine)
dim_date_df.head(7)

,date_id,order_date,qtr,month,year


##### 2) Perform stg_date left join DimDate tables to find new records 

In [16]:
# Merge on a composite key columns 
merged_date_df = stg_date_df.merge(dim_date_df, on = ['date_id']
                                 , how = 'left', indicator = True, suffixes = ('','_dim'))
merged_date_df.head()

,order_date,qtr,month,year,date_id,order_date_dim,qtr_dim,month_dim,year_dim,_merge
0,2003-02-24,1,2,2003,20030224,NaN,NaN,NaN,NaN,left_only
1,2003-05-07,2,5,2003,20030507,NaN,NaN,NaN,NaN,left_only
2,2003-07-01,3,7,2003,20030701,NaN,NaN,NaN,NaN,left_only
3,2003-08-25,3,8,2003,20030825,NaN,NaN,NaN,NaN,left_only
4,2003-10-10,4,10,2003,20031010,NaN,NaN,NaN,NaN,left_only


In [17]:
# Filter records that are only in staging but not in DimDate 
new_date_records_df = merged_date_df[merged_date_df['_merge']== 'left_only']
new_date_records_df.head()

,order_date,qtr,month,year,date_id,order_date_dim,qtr_dim,month_dim,year_dim,_merge
0,2003-02-24,1,2,2003,20030224,NaN,NaN,NaN,NaN,left_only
1,2003-05-07,2,5,2003,20030507,NaN,NaN,NaN,NaN,left_only
2,2003-07-01,3,7,2003,20030701,NaN,NaN,NaN,NaN,left_only
3,2003-08-25,3,8,2003,20030825,NaN,NaN,NaN,NaN,left_only
4,2003-10-10,4,10,2003,20031010,NaN,NaN,NaN,NaN,left_only


In [18]:
# drop columns from DimCustomer and keep only staging data
new_date_records_df= new_date_records_df[stg_date_df.columns]
new_date_records_df.head()

,order_date,qtr,month,year,date_id
0,2003-02-24,1,2,2003,20030224
1,2003-05-07,2,5,2003,20030507
2,2003-07-01,3,7,2003,20030701
3,2003-08-25,3,8,2003,20030825
4,2003-10-10,4,10,2003,20031010


##### 3) Insert new records into DimDate  MS Server SQL Table 

In [19]:
# re-order columns 
re_ordered_cols = ['date_id','order_date', 'qtr', 'month', 'year']
re_ordered_cols

['date_id', 'order_date', 'qtr', 'month', 'year']

In [20]:
new_date_records_df= new_date_records_df[re_ordered_cols]
new_date_records_df.head()

,date_id,order_date,qtr,month,year
0,20030224,2003-02-24,1,2,2003
1,20030507,2003-05-07,2,5,2003
2,20030701,2003-07-01,3,7,2003
3,20030825,2003-08-25,3,8,2003
4,20031010,2003-10-10,4,10,2003


In [21]:
# insert new records to the target table
total_inserted_salesdate_records = new_date_records_df.to_sql('DimDate', con = engine, if_exists = 'append', index = False)
print(total_inserted_salesdate_records)

252


In [22]:
new_date_records_df[new_date_records_df['date_id']==20030224]

,date_id,order_date,qtr,month,year
0,20030224,2003-02-24,1,2,2003
